### Async

In [ ]:
'use strict';

JavaScript has a runtime model based on an __event loop__, which is responsible for executing the code, collecting and processing events, and executing queued sub-tasks. 

It's async by design, but the API for developers evolved over years. 

First stop: callbacks.

In [ ]:
/*
 _    _      _                            _          _          _ _
| |  | |    | |                          | |        | |        | | |
| |  | | ___| | ___ ___  _ __ ___   ___  | |_ ___   | |__   ___| | |
| |/\| |/ _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \  | '_ \ / _ \ | |
\  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) | | | | |  __/ | |
 \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/  |_| |_|\___|_|_|
*/

{
    const { readFile, writeFile } = require('fs');

    function extractMetadata(sourceFile, destinationFile, callback) {
        
        readFile(sourceFile, { encoding: 'utf8' }, (err, data) => {

            if (err) {
                callback(err, null);
                return;
            }

            let pkg = null;
            try {
                pkg = JSON.parse(data);
            } catch (err) {

                // what if I want to throw an error, who catches it and when?
                callback(err, null);
                return;
            }

            const metadata = {
                pkgName: pkg.name,
                pkgVersion: pkg.version
            };

            writeFile(
                destinationFile,
                JSON.stringify(metadata, null, 4),
                { encoding: 'utf8' },
                (err) => {

                    if (err) {

                        // what if I want to throw an error, who catches it and when?
                        callback(err, null);
                        return;
                    }

                    callback(null, true);
                }
            )
        });
    }

    extractMetadata('./13-npm/package.json', './12-async-example-metadata.json', (err, result) => {

        if (err) {

            console.log('failed to write metadata file due to error:');
            console.log(err);

            return;
        }

        console.log('successfully wrote metadata file');
    });    
}

In the above example, the disadvantages are quite clear - nested callbacks make hard to properly handle the outcome of the operations, especially the propagation of errors because you have to use callbacks instead of `try { } catch () { }` blocks.

Second stop: Promises.

In [ ]:
{
    const { readFile, writeFile } = require('fs/promises');

    function extractMetadata(sourceFile, destinationFile) {

        return readFile(sourceFile, { encoding: 'utf8' })
            .then((data) => {

                let pkg = null;
                try {
                    pkg = JSON.parse(data);
                } catch (err) {
                    // what if I want to throw an error, who catches it and when?
                    return Promise.reject(err);
                }

                const metadata = {
                    pkgName: pkg.name,
                    pkgVersion: pkg.version
                };

                return writeFile(destinationFile, JSON.stringify(metadata, null, 4), { encoding: 'utf8' });
            })
            .catch((err) => {
                return Promise.reject(err);
            });
    }

    extractMetadata('./13-npm/package.json', './12-async-example-metadata.json')
        .then((data) => {
            console.log('successfully wrote metadata file');
        })
        .catch((err) => {

            console.log('failed to write metadata file due to error:');
            console.log(err);
        })
}

In the above example we chained Promises instead of nesting callbacks, but the outcome is still received through callbacks so it's not ideal (e.g. we still can't just throw errors and catch them elsewhere). Also, it's hard to follow the flow if you are not used to Promises and callbacks.

Last stop: async / await.

In [ ]:
{
    const { readFile, writeFile } = require('fs/promises');

    async function extractMetadata(sourceFile, destinationFile) {

        try {

            const data = await readFile(sourceFile, { encoding: 'utf8' });
            const pkg = JSON.parse(data);

            const metadata = {
                pkgName: pkg.name,
                pkgVersion: pkg.version
            };
            await writeFile(destinationFile, JSON.stringify(metadata, null, 4), { encoding: 'utf8' });

        } catch (err) {
            // we can just rethrow it, or wrap it in another error
            throw err;
        }
    }
    
    (async () => {

        try {

            await extractMetadata('./13-npm/package.json', './12-async-example-metadata.json');
            console.log('successfully wrote metadata file');

        } catch (err) {

            console.log('failed to write metadata file due to error:');
            console.log(err);
        }
    })();
}